In [1]:
import numpy as np
from BoomboxProcessor import BoomboxProcessor
import torch.nn as nn
import torch
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from torchmetrics.functional import accuracy as torch_acc
from sklearn.metrics import classification_report
import torch.optim as optim
from torch.utils.data import DataLoader
from train_encoding_model import BoomboxNet

/home/bsteinig/.pyenv/versions/3.10.6/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# First we need to trajectorize the data
import librosa
import glob
import pickle
from tqdm import tqdm

from MusicVectorizer import MusicVectorizer

SAMPLE_RATE = 16000

/home/bsteinig/.pyenv/versions/3.10.6/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mv = MusicVectorizer()

trajectories = dict()
for song in tqdm(glob.glob("transformed_audio/*.mp3")):
    song_data = librosa.load(song, sr=SAMPLE_RATE)[0]
    trajectories[song] = mv.trajectorize_song(song_data, SAMPLE_RATE)
    
np.save("data/salami_trajectories.npy", trajectories)

Running on device:  cuda


You are using a model of type mert_model to instantiate a model of type hubert. This is not supported for all configurations of models and can yield errors.
 15%|█▍        | 78/526 [02:44<08:48,  1.18s/it]

Song too short


 61%|██████    | 319/526 [11:25<07:27,  2.16s/it]

Song too short


 77%|███████▋  | 406/526 [14:07<03:27,  1.73s/it]

Song too short


100%|██████████| 526/526 [18:07<00:00,  2.07s/it]


In [2]:
data_folders = ['salami']
bx = BoomboxProcessor()

In [3]:
bx.load_trajectories(data_folders)

In [4]:
bx.load_encoding_model("models/model_50000.pt", BoomboxNet)

In [5]:
bx.encode_trajectories(device='cuda')

In [6]:
bx.split_encoded_trajectories(10,True)

In [7]:
songlet_trajectories = bx.get_songlet_trajectories()['salami']
timestep_trajectories = bx.get_trajectories()['salami']

In [8]:
songlet_lengths = dict()
for song in timestep_trajectories:
    time = (timestep_trajectories[song].shape[0] * 5) / 10

    songlet_lengths[song] = time

In [9]:
salami_ids = list()
for key in songlet_trajectories:
    salami_ids.append(key.split('/')[-1].split('.')[0])

In [10]:
from SegmentationLabels import SegmentationLabels
import os
import csv

In [11]:
sl = SegmentationLabels(salami_ids)

In [12]:
annotations = dict()
for salami_id in tqdm(salami_ids):
    annotations[salami_id] = sl.get_annotation(salami_id)

100%|██████████| 526/526 [00:00<00:00, 16476.38it/s]


In [13]:
NO_ANNO_DATA = []
for annotation in annotations:
    if len(annotations[annotation]) == 0:
        # remove the song from the dataset
        NO_ANNO_DATA.append(annotation)

66
226
300
292
859
804
182
102
96
301
256
263
174
375
246
872
67
259
1651
61
560
69
157
419
24


In [16]:
data = sl.get_annotation(61)
print(data)

[]
